##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Como migrar callbacks SessionRunHook para Keras

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/migrate/sessionrunhook_callback"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/guide/migrate/sessionrunhook_callback.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/guide/migrate/sessionrunhook_callback.ipynb"> <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"> Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/guide/migrate/sessionrunhook_callback.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>


No TensorFlow 1, para personalizar o comportamento do treinamento, você usa o `tf.estimator.SessionRunHook` com `tf.estimator.Estimator`. Este guia demonstra como migrar do `SessionRunHook` para callbacks personalizados do TensorFlow 2 com a API `tf.keras.callbacks.Callback`, que funciona com o Keras `Model.fit` para treinamento (assim como `Model.evaluate` e `Model.predict`). Você aprenderá como fazer isso implementando um `SessionRunHook` e uma tarefa `Callback` que mede exemplos por segundo durante o treinamento.

Exemplos de callbacks são o salvamento de checkpoints (`tf.keras.callbacks.ModelCheckpoint`) e a criação de resumos do [TensorBoard](%60tf.keras.callbacks.TensorBoard%60). Os [callbacks](https://www.tensorflow.org/guide/keras/custom_callback) do Keras são objetos que são chamados em diferentes pontos durante o treinamento/avaliação/previsão nas APIs integradas do Keras `Model.fit`/`Model.evaluate`/`Model.predict`. Você pode aprender mais sobre callbacks na documentação da API `tf.keras.callbacks.Callback`, bem como nos guias [Escrevendo seus próprios retornos de chamada](https://www.tensorflow.org/guide/keras/custom_callback.ipynb/) e [Treinamento e avaliação com os métodos integrados](https://www.tensorflow.org/guide/keras/train_and_evaluate) (na seção *Usando callbacks*).

## Configuração

Comece com os imports e um dataset simples para fins de demonstração:

In [ ]:
import tensorflow as tf
import tensorflow.compat.v1 as tf1

import time
from datetime import datetime
from absl import flags

In [ ]:
features = [[1., 1.5], [2., 2.5], [3., 3.5]]
labels = [[0.3], [0.5], [0.7]]
eval_features = [[4., 4.5], [5., 5.5], [6., 6.5]]
eval_labels = [[0.8], [0.9], [1.]]

## TensorFlow 1: crie um SessionRunHook personalizado com as APIs tf.estimator

Os exemplos do TensorFlow 1 a seguir mostram como configurar um `SessionRunHook` personalizado que mede exemplos por segundo durante o treinamento. Após criar o hook (`LoggerHook`), passe-o para o parâmetro `hooks` de `tf.estimator.Estimator.train`.

In [ ]:
def _input_fn():
  return tf1.data.Dataset.from_tensor_slices(
      (features, labels)).batch(1).repeat(100)

def _model_fn(features, labels, mode):
  logits = tf1.layers.Dense(1)(features)
  loss = tf1.losses.mean_squared_error(labels=labels, predictions=logits)
  optimizer = tf1.train.AdagradOptimizer(0.05)
  train_op = optimizer.minimize(loss, global_step=tf1.train.get_global_step())
  return tf1.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

In [ ]:
class LoggerHook(tf1.train.SessionRunHook):
  """Logs loss and runtime."""

  def begin(self):
    self._step = -1
    self._start_time = time.time()
    self.log_frequency = 10

  def before_run(self, run_context):
    self._step += 1

  def after_run(self, run_context, run_values):
    if self._step % self.log_frequency == 0:
      current_time = time.time()
      duration = current_time - self._start_time
      self._start_time = current_time
      examples_per_sec = self.log_frequency / duration
      print('Time:', datetime.now(), ', Step #:', self._step,
            ', Examples per second:', examples_per_sec)

estimator = tf1.estimator.Estimator(model_fn=_model_fn)

# Begin training.
estimator.train(_input_fn, hooks=[LoggerHook()])

## TensorFlow 2: crie um callback Keras personalizado para Model.fit

No TensorFlow 2, quando você usa o Keras `Model.fit` integrado (ou `Model.evaluate`) para treinamento/avaliação, poderá configurar um `tf.keras.callbacks.Callback` personalizado, que você passa para o parâmetro `callbacks` de `Model.fit` (ou `Model.evaluate`). (Saiba mais no guia [Como escrever seus próprios callbacks](../..guide/keras/custom_callback.ipynb).)

No exemplo abaixo, você escreverá um `tf.keras.callbacks.Callback` personalizado que registra várias métricas — ele medirá exemplos por segundo, que devem ser comparáveis ​​às métricas do exemplo anterior usando `SessionRunHook`.

In [ ]:
class CustomCallback(tf.keras.callbacks.Callback):

    def on_train_begin(self, logs = None):
      self._step = -1
      self._start_time = time.time()
      self.log_frequency = 10

    def on_train_batch_begin(self, batch, logs = None):
      self._step += 1

    def on_train_batch_end(self, batch, logs = None):
      if self._step % self.log_frequency == 0:
        current_time = time.time()
        duration = current_time - self._start_time
        self._start_time = current_time
        examples_per_sec = self.log_frequency / duration
        print('Time:', datetime.now(), ', Step #:', self._step,
              ', Examples per second:', examples_per_sec)

callback = CustomCallback()

dataset = tf.data.Dataset.from_tensor_slices(
    (features, labels)).batch(1).repeat(100)

model = tf.keras.models.Sequential([tf.keras.layers.Dense(1)])
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.05)

model.compile(optimizer, "mse")

# Begin training.
result = model.fit(dataset, callbacks=[callback], verbose = 0)
# Provide the results of training metrics.
result.history

## Próximos passos

Saiba mais sobre callbacks em:

- Documentação da API: `tf.keras.callbacks.Callback`
- Guia: [Escrevendo seus próprios callbacks](../..guide/keras/custom_callback.ipynb/)
- Guia: [Treinamento e avaliação com os métodos integrados](https://www.tensorflow.org/guide/keras/train_and_evaluate) (na seção *Usando callbacks* )

Você talvez também ache úteis os seguintes recursos relacionados à migração:

- O [Guia de migração de paradas antecipadas](early_stopping.ipynb) : `tf.keras.callbacks.EarlyStopping` é um callback integrado para paradas antecipadas (early stopping).
- O [Guia de migração do TensorBoard](tensorboard.ipynb): o TensorBoard permite rastrear e exibir métricas
- O [Guia de migração LoggingTensorHook e StopAtStepHook para callbacks Keras](logging_stop_hook.ipynb)